In [2]:
import os 
import tensorflow as tf 
from tensorflow.keras.preprocessing.image import ImageDataGenerator 
from tensorflow.keras import layers 
from tensorflow.keras import Model 
import matplotlib.pyplot as plt

from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau

In [3]:
base_dir = 'Fresh_Augmented'

In [4]:
# Add our data-augmentation parameters to ImageDataGenerator
image_datagen = ImageDataGenerator(rescale = 1./255., validation_split=0.2)


In [5]:
# Flow training images in batches of 20 using train_datagen generator
train_generator = image_datagen.flow_from_directory(base_dir, batch_size = 20, shuffle = True, subset ="training", class_mode = "categorical",  target_size = (224, 224))

# Flow validation images in batches of 20 using test_datagen generator
validation_generator = image_datagen.flow_from_directory(base_dir,  batch_size = 20, shuffle = True, subset ="validation", class_mode = "categorical", target_size = (224, 224))


Found 11749 images belonging to 12 classes.
Found 2933 images belonging to 12 classes.


In [6]:
from tensorflow.keras.applications.inception_v3 import InceptionV3
base_model = InceptionV3(input_shape = (224, 224, 3), include_top = False, weights = 'imagenet') 

In [7]:
for layer in base_model.layers:
    layer.trainable = True

In [8]:
# Flatten the output layer to 1 dimension
x = layers.Flatten()(base_model.output)

# Add a fully connected layer with 512 hidden units and ReLU activation
x = layers.Dense(512, activation='relu')(x)

# Add a dropout rate of 0.5
x = layers.Dropout(0.5)(x)

# Add a final sigmoid layer for classification
x = layers.Dense(12, activation='softmax')(x)

model = tf.keras.models.Model(base_model.input, x)

model.compile(optimizer = tf.keras.optimizers.RMSprop(lr=0.0001), loss = 'categorical_crossentropy',metrics = ['acc'])


In [9]:
os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'

In [10]:
# Checkpoints which saves partial model at each epoch and stops the model if validatation loss isn't changing
checkpoint = ModelCheckpoint("saved_model_temp.h5", monitor="val_loss", mode="min", save_best_only = True, verbose=1)
earlystop = EarlyStopping(monitor = 'val_loss', min_delta = 0, patience = 3, verbose = 1, restore_best_weights = True)
reduce_lr = ReduceLROnPlateau(monitor = 'val_loss',factor = 0.2, patience = 3, verbose = 1, min_delta = 0.00001)
callbacks = [earlystop, checkpoint, reduce_lr]

In [11]:
# Training model
vgghist = model.fit(train_generator, validation_data = validation_generator, steps_per_epoch = 11749//20 , epochs = 20, callbacks = callbacks)

Epoch 1/20
587/587 [==============================] - 137s 205ms/step - loss: 0.8882 - acc: 0.8006 - val_loss: 4.5402 - val_acc: 0.8970

Epoch 00001: val_loss improved from inf to 4.54019, saving model to saved_model_temp.h5
Epoch 2/20
587/587 [==============================] - 114s 194ms/step - loss: 0.1422 - acc: 0.9785 - val_loss: 6.0808 - val_acc: 0.9011

Epoch 00002: val_loss did not improve from 4.54019
Epoch 3/20
587/587 [==============================] - 117s 199ms/step - loss: 0.1115 - acc: 0.9835 - val_loss: 41.2849 - val_acc: 0.8016

Epoch 00003: val_loss did not improve from 4.54019
Epoch 4/20
587/587 [==============================] - 114s 195ms/step - loss: 0.1115 - acc: 0.9857 - val_loss: 8.1831 - val_acc: 0.8762
Restoring model weights from the end of the best epoch.

Epoch 00004: val_loss did not improve from 4.54019

Epoch 00004: ReduceLROnPlateau reducing learning rate to 1.9999999494757503e-05.
Epoch 00004: early stopping


In [13]:
import time
t = time.time()

# Exporting Model in .pb format
export_path = "Modified_Dataset/saved_models/{}_Inceptionv3".format(int(t))
model.save(export_path)

export_path

INFO:tensorflow:Assets written to: Modified_Dataset/saved_models/1621593225_Inceptionv3\assets


'Modified_Dataset/saved_models/1621593225_Inceptionv3'

In [14]:
#Converting Model to tflite optimised model 
converter = tf.lite.TFLiteConverter.from_saved_model(export_path)

tflite_model = converter.convert()
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.target_spec.supported_types = [tf.float16]
tflite_quantized_model = converter.convert()
open(export_path + "/float16_optimised_model_Inceptionv3.tflite", "wb").write(tflite_quantized_model)

96037296

In [15]:
#Converting Model to tflite model
converter = tf.lite.TFLiteConverter.from_saved_model(export_path)

# tflite_model = converter.convert()
# converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.target_spec.supported_types = [tf.float16]
tflite_quantized_model = converter.convert()
open(export_path + "/float16_model_Inceptionv3.tflite", "wb").write(tflite_quantized_model)

191993756